### Задание 3 - Индекс | Булев поиск

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet') 
nltk.download('punkt')

In [2]:
import os
from bs4 import BeautifulSoup

In [7]:
# словарь (номер документа -> контент (заголовок + содержимое статьи))
content = {}
i = 1
for file_ in os.listdir('links'):
  if (file_.__contains__('.html')):
    path_ = os.path.join('links', file_)
    file_text = open(path_).read()
  
    soup = BeautifulSoup(file_text)
    
    page_content = []
    for text in soup.find('h1', {'class': 'tm-article-snippet__title tm-article-snippet__title_h1'}):
      title = text.get_text()
      page_content.append(title)

    data = soup.find('div', {'xmlns': 'http://www.w3.org/1999/xhtml'})
    for p in data.find_all('p'):
       text = p.get_text()
       page_content.append(text)
    
    content[i] = page_content
    i += 1

In [18]:
# пример текста 3 документа
content[3]

['Российские ученые спрогнозировали падение на Луну космического объекта',
 'По данным исследователей из Института наукоемких технологий и передовых материалов Дальневосточного федерального университета и Института прикладной астрономии Российской академии наук, на Луну упадет объект WE0913A. Ученые прогнозируют его столкновение с обратной стороной спутника Земли 4 марта.',
 'Исследователи определили WE0913A как отработавшую ступень ракеты. Они говорят, что объект столкнется с Луной в 12:34 мск. Столб пыли, вызванный ударом, будет возможно наблюдать с Земли.',
 'Тело упадет со скоростью 2,6 тыс. м/c, что позволит увидеть данное явление даже невооруженным глазом. Зрение человека способно различать объекты на расстоянии до шестой звездной величины, а столкновение WE0913A с Луной произойдет в двух. Студент Института наукоемких технологий и передовых материалов ДВФУ Тимур Ноздрачев сообщил о готовности исследователей обсерваторий Италии, Словакии и Таджикистана провести наблюдения падения 

In [11]:
import string
from nltk.corpus import stopwords
import re

In [22]:
def tokenize(content):
    result = []
    tokens = set() 
    for c in content:
      tokens_ = nltk.word_tokenize(c)
      tokens_ = [i.lower() for i in tokens_]
      tokens_ = [i for i in tokens_ if ( i not in string.punctuation )]
      tokens_ = filter(None, [re.sub(r"[^a-zа-я-]+", r"", i) for i in tokens_])

      stop_words = stopwords.words('russian')
      stop_words.extend(['что', 'это', 'так', 'вот', 'как', 'в', 'к',
                         'на', 'о', 'при', 'из-за', 'за', 'ао', 'но', 'х',
                         'хотя', 'среди', 'помимо', 'с'])
      tokens_ = [i for i in tokens_ if ( i not in stop_words )]

      stop_words2 = stopwords.words('english')
      stop_words2.extend(['the', 'e', 'a', 'd', 'b', 'x', 'c'])
      tokens_ = [i for i in tokens_ if ( i not in stop_words2 )]

      tokens_ = [i.replace("«", "").replace("»", "") for i in tokens_]

      for t_ in tokens_:
        tokens.add(t_)
      result += [tokens]

    return result

In [ ]:
index={}
for i in range(1, 101):
    data = []
    for c in tokenize(content[i]):
        data += c
    data = list(dict.fromkeys(data))
    for token in data:
        if token in index.keys():
            index[token].append(i)
        else:
            index[token] = [i]

In [27]:
with open('inverted_index.txt', "w", encoding="utf-8") as f:
    for key in index:
        f.write('{\"count\":' + str(len(index[key])) + ',\"inverted_array\":' + str(index[key]) + ',\"word\":\"' + key + '\"}\n')

In [29]:
from google.colab import files
files.download('inverted_index.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Реализация булева поиска

In [45]:
def preprocess_for_search(content):
    result = []
    tokens = set() 
    
    tokens_ = nltk.word_tokenize(content)
    tokens_ = [i.lower() for i in tokens_]
    tokens_ = [i for i in tokens_ if ( i not in string.punctuation )]
    tokens_ = filter(None, [re.sub(r"[^a-zа-я-]+", r"", i) for i in tokens_])

    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'как', 'в', 'к',
                         'на', 'о', 'при', 'из-за', 'за', 'ао', 'но', 'х',
                         'хотя', 'среди', 'помимо', 'с'])
    tokens_ = [i for i in tokens_ if ( i not in stop_words )]

    stop_words2 = stopwords.words('english')
    stop_words2.extend(['the', 'e', 'a', 'd', 'b', 'x', 'c'])
    tokens_ = [i for i in tokens_ if ( i not in stop_words2 )]

    tokens_ = [i.replace("«", "").replace("»", "") for i in tokens_]

    for t_ in tokens_:
      tokens.add(t_)
    result += [tokens]

    return result

In [30]:
import json

In [113]:
# Текст - Тело упадет со скоростью 2,6 тыс. м/c (из 3 документа)
text = input()
tokens_search = []
for t in preprocess_for_search(text):
    tokens_search += t

In [114]:
tokens_search

['упадет', 'тело', 'скоростью']

In [115]:
# поиск пересечения нужных слов и слов из созданного индекса
search = {}
for token in tokens_search:
    if token in index.keys():
        search[token] = index[token]
    else:
        search[token] = []

In [116]:
search

{'скоростью': [3, 75, 77], 'тело': [3, 88], 'упадет': [3]}

In [128]:
def bool_search(search_type, search):
  i = 1
  document = []
  for token in search:
      if i == 1:
        document = search[token]
      else:
        if search_type == 'AND':
           # все токены должны быть в документе
          document = list(set(document) & set(search[token]))
        if search_type == 'OR':
           # достаточно хотя бы одного токена в документе
          document = list(set(document) | set(search[token]))
      i += 1
  print(document)

In [129]:
bool_search('AND', search)

[3]


In [130]:
bool_search('OR', search)

[3, 88, 75, 77]
